In [39]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from math import ceil,log2
import random
import torchvision
from torchvision import transforms
# from model import generator, discriminator

In [106]:
class generator(nn.Module):

    def __init__(self, encoding_dims = 100, step_channels = 6
                 , out_channels = 3):
        super(G, self).__init__()
        
        nz = encoding_dims
        ngf = step_channels
        nc = out_channels
        
        self.conv1 = nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0)
        self.bn1 = nn.BatchNorm2d(ngf * 8)

        self.conv2 = nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(ngf * 4)

        self.conv3 = nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ngf * 2)

        self.conv4 = nn.ConvTranspose2d(ngf * 2, ngf * 1, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(ngf * 1)

        self.conv5 = nn.ConvTranspose2d(ngf * 1, nc, 4, 2, 1)

        self.relu = nn.ReLU(True)
        self.tanh = nn.Tanh()

        self.__initialize_weights()

    def forward(self, input):
        x = input.view(input.size(0), -1, 1, 1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)

        x = self.conv5(x)
        output = self.tanh(x)
        return output
    
def __initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, 0.02)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.normal_(1.0, 0.02)
                m.bias.data.fill_(0)


# class generatora(nn.Module):
  
#   def __init__(
#       self,
#       encoding_dims = 100,
#       out_size = 32,
#       out_channels = 3,
#       step_channels = 64,
#       nonlinearity = None,
#       last_nonlinearity = None
#   ):
#     super().__init__()
    
#     if out_size < 16 or ceil(log2(out_size)) != log2(out_size):
#             raise Exception(
#                 "Target Image Size must be at least 16*16 and an exact power of 2"
#             )
        
#     num_repeats = out_size.bit_length() - 4
#     d = (step_channels*(2**num_repeats))
    
#     self.encoding_dims = encoding_dims

#     nl = nn.LeakyReLU(.2) if nonlinearity is None else nonlinearity
#     last_nl = nn.Tanh() if last_nonlinearity is None else last_nonlinearity
    
#     model =[]
    
#     model.append(
#         nn.Sequential(
#           nn.ConvTranspose2d(encoding_dims, d, 4, 1, 0),
#           nn.BatchNorm2d(d),
#           nl
#         )
#     )
    
#     for i in range(num_repeats):
#       model.append(
#           nn.Sequential(
#           nn.ConvTranspose2d(d, d//2, 4, 2, 1),
#           nn.BatchNorm2d(d//2),
#           nl
#           )
#       )
      
#       d = d//2
    
    
#     model.append(
#         nn.Sequential(
#             nn.ConvTranspose2d(step_channels, out_channels, 4, 2, 1, bias = True),
#             last_nl
#         )
#     )
    
#     self.model = nn.Sequential(*model)
#     self._weight_initializer()
    
#   def forward(self,x):
    
#     x = x.view(-1, self.encoding_dims, 1, 1)
#     return self.model(x)
  
  
#   def _weight_initializer(self):
#         r"""Default weight initializer for all generator models.
#         Models that require custom weight initialization can override this method
#         """
#         for m in self.modules():
#             if isinstance(m, nn.ConvTranspose2d):
#                 nn.init.kaiming_normal_(m.weight)
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1.0)
#                 nn.init.constant_(m.bias, 0.0)  

In [107]:
class discriminator(nn.Module):

    def __init__(self, ndf, nc, num_classes=10):
        super(D, self).__init__()
        
        ndf = step_channels
        nc = in_channels
        
        
        self.ndf = ndf
        self.lrelu = nn.LeakyReLU(0.2, inplace=True)
        self.conv1 = nn.Conv2d(nc, ndf, 4, 2, 1)
        self.conv2 = nn.Conv2d(ndf, ndf * 2, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(ndf * 2)
        self.conv3 = nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(ndf * 8)
        self.conv5 = nn.Conv2d(ndf * 8, ndf * 1, 4, 1, 0)
        self.gan_linear = nn.Linear(ndf * 1, 1)
        self.aux_linear = nn.Linear(ndf * 1, num_classes)

        self.sigmoid = nn.Sigmoid()
        self.__initialize_weights()
    
    def forward(self, input):

        x = self.conv1(input)
        x = self.lrelu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.lrelu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.lrelu(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.lrelu(x)

        x = self.conv5(x)
        x = x.view(-1, self.ndf * 1)
        c = self.aux_linear(x)

        s = self.gan_linear(x)
        s = self.sigmoid(s)
        return s.squeeze(1), c.squeeze(1)

    def __initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, 0.02)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.normal_(1.0, 0.02)
                m.bias.data.fill_(0)
                

# class discriminatora(nn.Module):
  
#     def __init__(
#         self,
#         num_classes,
#         in_size = 32,
#         in_channels = 3,
#         step_channels = 64,
#         nonlinearity = None,
#         last_nonlinearity = None
#     ):
    
#         super().__init__()
    
#         if in_size < 16 or ceil(log2(in_size)) != log2(in_size):
#             raise Exception(
#                 "Input Image Size must be at least 16*16 and an exact power of 2"
#             )
    
#         self.num_classes = num_classes
#         num_repeats = in_size.bit_length() - 4
#         d = step_channels 
#         model = []

#         nl = nn.LeakyReLU(.2) if nonlinearity is None else nonlinearity
#         last_nl = nn.LeakyReLU(.2) if last_nonlinearity is None else last_nonlinearity

#          model.append(
#             nn.Sequential(
#                 nn.Conv2d(in_channels, d, 4, 2, 1, bias = True),
#                 nl)
#         )

#         for i in range(num_repeats):

#             model.append(
#                 nn.Sequential(
#                     nn.Conv2d(d, d*2, 4, 2, 1, bias = False),
#                      nn.BatchNorm2d(d*2),
#                      nl
#                 )
#             )

#              d = d*2


#         self.disc = nn.Sequential(
#                 nn.Conv2d(d, 1, 4, 1, 0, bias = True),
#                 nn.Sigmoid()
#         )

#         self.aux = nn.Sequential(
#                 nn.Conv2d(d, self.num_classes, 4, 1, 0, bias = True),
#                 last_nl
#         )

      
#         self.model = nn.Sequential(*model)
#         self._weight_initializer()
          
  
#     def forward(self, x):
        
#         x = self.model(x)
#         dx = self.disc(x)
#         ax = self.aux(x)
        
#         return dx.view(-1), ax.view(-1, self.num_classes) 
        
#     def _weight_initializer(self):
#         r"""Default weight initializer for all disciminator models.
#         Models that require custom weight initialization can override this method
#         """
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight)
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1.0)
#                 nn.init.constant_(m.bias, 0.0)

In [108]:
device = 'cuda:0' if torch.cuda.torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [110]:
class parser():
  def __init__(self):
    self.num_classes = 10
    self.encoding_dims = 100
    self.batch_size = 64
    self.epochs = 4
    self.image_size = 64
    self.num_workers = 4
    self.channels = 1
    self.step_channels = 64

In [111]:
parser = parser()

In [96]:
generator = generator(encoding_dims = parser.encoding_dims, out_size = parser.image_size, out_channels = parser.channels, step_channels = parser.step_channels).to(device)
discriminator = discriminator(num_classes = parser.num_classes, in_size = parser.image_size, in_channels = parser.channels, step_channels = parser.step_channels).to(device)

TypeError: forward() got an unexpected keyword argument 'encoding_dims'

In [97]:
transform=transforms.Compose([
    transforms.Resize(parser.image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = torchvision.datasets.MNIST(root='./data', download=True, train=True, transform=transform)

In [98]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=parser.batch_size, shuffle=True, num_workers=parser.num_workers) 

In [99]:
BCEloss = nn.BCELoss().to(device)
CEPloss = nn.CrossEntropyLoss().to(device)

In [100]:
optimG = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimD = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [101]:
embedding = nn.Embedding(parser.num_classes,parser.encoding_dims).to(device)
fixed_labels = torch.LongTensor([range(parser.num_classes)]*10).view(-1).to(device)
fixed_noise_init = torch.Tensor(100, parser.encoding_dims).normal_(0,1).to(device)
fixed_noise = torch.mul(fixed_noise_init, embedding(fixed_labels))

In [102]:
for epoch in range(parser.epochs):
  
  for i, data in enumerate(dataloader):
    
    print('done')
    data_input, label = data
    
    #######################
    # real input and label
    #######################
   
    real_input = data_input.to(device)
    real_label = label.to(device)
    real_ = torch.ones((parser.batch_size)).to(device)
    
    #######################
    # fake input and label
    #######################
    
    noise_init = torch.Tensor(parser.batch_size, parser.encoding_dims).to(device)
    fake_labels = torch.torch.LongTensor(parser.batch_size).random_(parser.num_classes).to(device)
    noise = torch.mul(noise_init, embedding(fake_labels)).view(parser.batch_size, parser.encoding_dims, 1, 1).to(device)
    fake_input = generator(noise)
    fake_ = torch.zeros((parser.batch_size)).to(device)
    
    #######################
    # update discriminator
    #######################
    
    discriminator.zero_grad()
    
    real_prediction, real_class = discriminator(real_input)
    fake_prediction, fake_class = discriminator(fake_input)
    
    real_loss = BCEloss(real_prediction, real_) + CEPloss(real_class, real_label)*10
    fake_loss = BCEloss(fake_prediciton, fake_) + CEPloss(fake_class, fake_labels)*10
    
    D_loss = real_loss + fake_loss
    D_loss.backward()
    
    optimD.step()
    
    #######################
    # update generator
    #######################
    
    generator.zero_grad()
    
    fake_predicition, fake_class = discriminator(fake_input)
    
    loss = BCEloss(fake_prediction, real_) + CEPloss(fake_class, fake_labels)*10
    loss.backward()
    optimG.step()
    

done


RuntimeError: Assertion `x >= 0. && x <= 1.' failed. input value should be between 0~1, but got -nan at /pytorch/aten/src/THNN/generic/BCECriterion.c:62